In [8]:
import pandas as pd
from rnnoise_wrapper import RNNoise
from pydub import AudioSegment

In [3]:
train = pd.read_csv('data/train_gt.csv', header=None, names=['names', 'labels'])

In [ ]:
for i in train['names'].tolist():

    audio = AudioSegment.from_mp3(f'data/train/{i}')

    if audio.frame_rate!=48000:
        audio.set_frame_rate(48000)
    if audio.sample_width!=2:
        audio.set_sample_width(2)
    if audio.channels!=1:
        audio.set_channels(1)

    denoiser = RNNoise()

    denoised_audio = denoiser.filter(audio)
    denoised_audio.export(f'data/train_final/{i}', format="mp3")